In [1]:
pip install gitpython ipywidgets


Defaulting to user installation because normal site-packages is not writeable

   ---------------------------------------- 0/4 [webcolors]
   ---------- ----------------------------- 1/4 [uri-template]
   ------------------------------ --------- 3/4 [isoduration]
   ------------------------------ --------- 3/4 [isoduration]
   ---------------------------------------- 4/4 [isoduration]

Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import shutil
import subprocess
from pathlib import Path
from git import Repo, InvalidGitRepositoryError, GitConfigParser
import ipywidgets as widgets
from IPython.display import display
import atexit
from urllib.parse import urlparse, unquote

repo = None
repo_path = None
cleanup_registered = False

def delete_git_folder():
    global repo_path
    if repo_path and Path(repo_path, ".git").exists():
        try:
            shutil.rmtree(Path(repo_path, ".git"))
            print("🧹 .git folder deleted on session exit.")
        except Exception as e:
            print(f"⚠️ Cleanup failed: {e}")

def push():
    global repo, repo_path, cleanup_registered

    url_input = widgets.Text(
        value='',
        placeholder='Paste full Jupyter notebook URL here',
        description='Notebook URL:',
        layout=widgets.Layout(width='100%')
    )

    commit_msg = widgets.Text(
        value='Pushed from notebook',
        description='Message:',
        layout=widgets.Layout(width='100%')
    )

    remote_url_input = widgets.Text(
        value='',
        description='Remote:',
        placeholder='https://github.com/user/repo.git',
        layout=widgets.Layout(width='100%')
    )

    user_name_input = widgets.Text(
        value='',
        description='Git Name:',
        layout=widgets.Layout(width='50%')
    )

    user_email_input = widgets.Text(
        value='',
        description='Git Email:',
        layout=widgets.Layout(width='50%')
    )

    push_btn = widgets.Button(description="Push Now 🚀", button_style='success')
    display(url_input, remote_url_input, user_name_input, user_email_input, commit_msg, push_btn)

    def check_git_user_config():
        try:
            name = subprocess.check_output(['git', 'config', '--global', 'user.name']).decode().strip()
            email = subprocess.check_output(['git', 'config', '--global', 'user.email']).decode().strip()
            return name, email
        except subprocess.CalledProcessError:
            return None, None

    def configure_git_user(name, email):
        subprocess.run(['git', 'config', '--global', 'user.name', name])
        subprocess.run(['git', 'config', '--global', 'user.email', email])

    def on_push_clicked(b):
        global repo_path, repo, cleanup_registered

        full_url = url_input.value.strip()
        if not full_url.startswith("http"):
            print("❌ Invalid URL. Please paste a full notebook URL.")
            return

        parsed = urlparse(full_url)
        notebook_path = unquote(parsed.path).replace("/notebooks/", "", 1)
        notebook_file = Path(notebook_path)

        if not notebook_file.exists():
            print(f"❌ Notebook file not found: {notebook_file}")
            return

        repo_path = str(notebook_file.parent)
        os.chdir(repo_path)

        # Step 1: Check Git user config
        name, email = check_git_user_config()
        if not name or not email:
            name = user_name_input.value.strip()
            email = user_email_input.value.strip()
            if not name or not email:
                print("❌ Git user.name and user.email are required.")
                return
            configure_git_user(name, email)
            print(f"✅ Git identity set: {name} <{email}>")

        # Step 2: Init Git if needed
        if not (Path(repo_path) / ".git").exists():
            subprocess.run(["git", "init"], cwd=repo_path)
            print("⚙️ Initialized Git repository.")

        try:
            repo = Repo(repo_path)
        except InvalidGitRepositoryError:
            print("❌ Invalid Git repository.")
            return

        # Step 3: Set remote URL
        remote_url = remote_url_input.value.strip()
        if remote_url:
            try:
                subprocess.run(["git", "remote", "remove", "origin"], cwd=repo_path, stderr=subprocess.DEVNULL)
            except:
                pass
            subprocess.run(["git", "remote", "add", "origin", remote_url], cwd=repo_path)
            print(f"🔗 Remote set to: {remote_url}")

        # Step 4: Commit & Push
        try:
            if "main" not in repo.heads:
                repo.git.checkout("-b", "main")  # create main if it doesn't exist
            else:
                repo.git.checkout("main")        # switch to main if it exists

            repo.git.add(notebook_file.name)
            repo.git.commit("-m", commit_msg.value.strip())

            # Set upstream only on first push
            repo.git.push("--set-upstream", "origin", "main")
            print(f"✅ {notebook_file.name} pushed to 'main' branch successfully.")
        except Exception as e:
            print(f"❌ Push failed: {e}")

        if not cleanup_registered:
            atexit.register(delete_git_folder)
            cleanup_registered = True

    push_btn.on_click(on_push_clicked)

In [5]:
push()

Text(value='', description='Notebook URL:', layout=Layout(width='100%'), placeholder='Paste full Jupyter noteb…

Text(value='', description='Remote:', layout=Layout(width='100%'), placeholder='https://github.com/user/repo.g…

Text(value='', description='Git Name:', layout=Layout(width='50%'))

Text(value='', description='Git Email:', layout=Layout(width='50%'))

Text(value='Pushed from notebook', description='Message:', layout=Layout(width='100%'))

Button(button_style='success', description='Push Now 🚀', style=ButtonStyle())

❌ Notebook file not found: _Jupyter_notebook\git_helper v1.0.ipynb
